In [1]:
!pip install flask-ngrok pyngrok


In [2]:
!pip install flask openai-whisper yt-dlp
!sudo apt update && sudo apt install ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 19.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 7.1 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=8d35804f88a68443669a525cddeb72bfb7ce65f19b9d4b2d6572fa1c11ae1233
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper
Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https

In [3]:
!pip install flask-cors

Update for youtube to chat to talk with AI

In [4]:
# !pip install -U -q google-generativeai

Documentation app

In [5]:
!pip install sumy transformers torch sentencepiece

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 68.0 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21695 sha256=cd2abc1591d615400ee5f2120692af0004bf0f471b619072245d6e058be65423
  Stored in directory: /root/.cache/pip/wheels/32/99/64/59305409cacd03aa03e7bddf31a9db34b1fa7033bd41972662
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=fdd59ebde4005462e33e0ced681e3bdef2122a2095e8034b85f91cb75921266e
  Stored in directory: /root/.cache/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built breadability docopt


In [6]:
# !pip install -U google-generativeai

New version for AIDocu(untested with app but recommended)

Stable version(have test and work, the cookie seems to stay stable)

1) Vid2Chat

In [8]:
import os
import torch
import tempfile
import threading
import logging
import shutil
import whisper
import yt_dlp
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
from google import genai
from google.colab import userdata

# --- 1. Configuration ---
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}})

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
gpu_lock = threading.Lock()

# Global Model Placeholders
whisper_model = None
client = None  # This replaces chat_model for the new SDK

# --- 2. Model Loading ---
def load_all_models():
    global whisper_model, client

    print(f"--- Loading Models on {DEVICE} ---")

    # Whisper
    try:
        whisper_model = whisper.load_model("base", device=DEVICE)
        print("✅ Whisper Model Loaded")
    except Exception as e:
        print(f"❌ Whisper Load Error: {e}")

    # Gemini AI (Fixed for google.genai SDK)
    try:
        api_key = userdata.get('MY_GEMINI_API_KEY')
        # Use the Client class - no .configure() needed in this SDK
        client = genai.Client(api_key=api_key)
        print("✅ Gemini AI Client Initialized")
    except Exception as e:
        print(f"❌ Gemini Key missing or Client error! Error: {e}")

# --- 3. Logic Functions ---
def get_video_info(url):
    tmpdir = tempfile.mkdtemp()
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best',
        'outtmpl': os.path.join(tmpdir, 'video.mp4'),
        'quiet': True
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        return info.get('title', 'Video'), os.path.join(tmpdir, 'video.mp4'), tmpdir

# --- 4. API Routes ---

@app.route('/process_full_video', methods=['POST'])
def process_full_video():
    data = request.get_json()
    url = data.get("url")
    if not url:
        return jsonify({"error": "No URL provided"}), 400

    with gpu_lock:
        try:
            logger.info(f"Downloading video from: {url}")
            title, path, tmpdir = get_video_info(url)

            logger.info("Starting transcription...")
            result = whisper_model.transcribe(path, fp16=torch.cuda.is_available())
            transcript = result.get("text", "").strip()

            shutil.rmtree(tmpdir)
            return jsonify({"video_name": title, "transcript": transcript})
        except Exception as e:
            logger.error(f"Error processing video: {str(e)}")
            return jsonify({"error": str(e)}), 500

@app.route('/chat', methods=['POST'])
def chat():
    data = request.get_json()
    user_message = data.get("message")
    context_transcript = data.get("transcript")
    history = data.get("history", [])

    if not user_message:
        return jsonify({"error": "No message provided"}), 400

    memory_block = "\n".join([f"{m['role'].upper()}: {m['content']}" for m in history]) if history else "No history yet."

    full_prompt = (
        f"SYSTEM: You are a helpful assistant with conversation memory.\n"
        f"TRANSCRIPT:\n{context_transcript}\n\n"
        f"MEMORY:\n{memory_block}\n\n"
        f"USER: {user_message}\n"
    )

    try:
        if client is None:
            return jsonify({"error": "Gemini client not initialized"}), 500

        # Updated syntax for google-genai 2.0+
        response = clien0t.models.generate_content(
            model='gemini-2.5-flash-lite',
            contents=full_prompt
        )
        return jsonify({"response": response.text})
    except Exception as e:
        logger.error(f"Chat error: {str(e)}")
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    load_all_models()
    NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

    tunnels = ngrok.get_tunnels()
    for t in tunnels:
        ngrok.disconnect(t.public_url)

    public_url = ngrok.connect(5001, bind_tls=True, domain="notably-valid-bunny.ngrok-free.app")
    print(f"\n🚀 Vid2Chat API URL: {public_url}\n")
    app.run(port=5001, threaded=True)

--- Loading Models on cpu ---
✅ Whisper Model Loaded
✅ Gemini AI Client Initialized

🚀 Vid2Chat API URL: NgrokTunnel: "https://notably-valid-bunny.ngrok-free.app" -> "http://localhost:5001"

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5001
INFO:werkzeug:Press CTRL+C to quit
